In [3]:
import torch

In [32]:
B = 1 # keep 1
H = 1 # keep 1
N = 512
D = 64
DV = 64

In [33]:
torch.random.manual_seed(42)
q = (torch.randn((B, H, N, D), dtype=torch.bfloat16, device='cpu')/float(D)**.5).to(torch.float32)
k = (torch.randn((B, H, N, D), dtype=torch.bfloat16, device='cpu')/float(D)**.5).to(torch.float32)
v = (torch.randn((B, H, N, DV), dtype=torch.bfloat16, device='cpu')/DV).to(torch.float32)
A = torch.rand_like(v).to(torch.float32)
q.requires_grad = True
k.requires_grad = True
v.requires_grad = True

In [34]:
attn = q @ k.transpose(-2, -1)
attn.masked_fill_(~torch.tril(torch.ones(q.shape[-2], q.shape[-2], dtype=torch.bool, device=q.device)), 0)
o = attn @ v

In [35]:
J = (o*A).sum()
J.backward()

In [36]:
q_grad = q.grad
k_grad = k.grad
v_grad = v.grad

In [37]:
torch.random.manual_seed(42)
q = (torch.randn((B, H, N, D), dtype=torch.bfloat16, device='cpu') / float(D)**.5).to(torch.float32)
k = (torch.randn((B, H, N, D), dtype=torch.bfloat16, device='cpu') / float(D)**.5).to(torch.float32)
v = (torch.randn((B, H, N, DV), dtype=torch.bfloat16, device='cpu') / DV).to(torch.float32)
A = torch.rand_like(v).to(torch.float32)
q.requires_grad = True
k.requires_grad = True
v.requires_grad = True

attn = q @ k.transpose(-2, -1)
attn.masked_fill_(~torch.tril(torch.ones(q.shape[-2], q.shape[-2], dtype=torch.bool, device=q.device)), 0)
o = attn @ v

# Compute the loss
J = (o * A).sum()

# Call backward with retain_graph=True to preserve the graph
J.backward(retain_graph=True)

# Compute gradient of J w.r.t. o explicitly
grad_o = torch.autograd.grad(J, o, retain_graph=True)[0]

# here, access to q.grad, k.grad, v.grad, o.grad

In [38]:
torch.allclose(q_grad, q.grad), torch.allclose(k_grad, k.grad), torch.allclose(v_grad, v.grad)

(True, True, True)

In [39]:
torch.allclose(grad_o, A)

True